## effect size sharing for multi-gene qtls

say we have a qtl that impacts multiple genes. How does the fractional effect of that qtl on each gene vary? w.r.t for instance distance from qtl to tss? maybe tie in ABC connections here? Maybe build a model that predicts how qtl impact on each gene will be split based on gene baseline expression, distance, ect

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import os
import ast
import re

import sys
sys.path.append('/home/klawren/oak/pcqtls/workflow/scripts')
from notebook_helper_functions import *

import upsetplot as up
from tqdm.auto import tqdm  # for notebooks

# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

In [2]:
# get outputs from a config file
prefix = '/home/klawren/oak/pcqtls'
import yaml
config_path= f'{prefix}/config/proteincoding_main.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# load in the tissue ids 
tissue_df = load_tissue_df(config)
tissue_ids = list(tissue_df['Tissue'])

# select just 1 tissue id to do for now 
my_tissue_id = 'Cells_Cultured_fibroblasts' # because only one cell type
#tissue_ids = [tissue_id]

### load in data
effect sizes for qtls
overlap dfs
gencode for tss starts

In [54]:
e_nominal = load_e_nominal(config, my_tissue_id, chr_id=22)
pc_nominal = load_pc_nominal(config, my_tissue_id, chr_id=22)
overlap = load_overlap(config, my_tissue_id)
pc_susie = load_pc_susie(config, my_tissue_id)

# set e_nominal to be index based on variant-cluster pairs
e_nominal_cid = e_nominal.set_index('var_cluster')

In [97]:
# look at just one chr to start
pc_susie['chr'] = pc_susie['variant_id'].str.split('chr').str[1].str.split('_').str[0]
pc_susie = pc_susie[pc_susie['chr'] == '22']

take the subset of varaints in pcqtl determined credible sets, and calculate the varience-explained on a per gene basis for each gene in the cluster and for the cluster-pc overall

variance for each egene-cluster-credibleset(pc and cs num)

In [119]:
pc_susie['var_cluster'].iloc[0]

'chr22_18933556_A_G_b38_ENSG00000100033.16_ENSG00000183628.12'

In [106]:
# get a subset of variants that ended up in credible sets
e_nominal_pcsusie_subset = e_nominal[e_nominal['var_cluster'].isin(pc_susie['var_cluster'])]

# I want to get a pip weighted variance for each egene
e_nominal_pcsusie_subset['var_cluster_egene'] = e_nominal_pcsusie_subset['variant_id'] + '_' + e_nominal_pcsusie_subset['phenotype_id']

# merge in the data from the susie finemapping
e_nominal_pcsusie_subset = pd.merge(e_nominal_pcsusie_subset, pc_susie[['cs_id', 'var_cluster', 'pip', 'pc_num', 'lead_variant_id', 'num_vars']], on='var_cluster', how='right')

# pip weighted variance
e_nominal_pcsusie_subset['variance'] = e_nominal_pcsusie_subset['slope'].apply(np.square) * 100
e_nominal_pcsusie_subset['variance_weighted'] = e_nominal_pcsusie_subset['variance'] * e_nominal_pcsusie_subset['pip']

# we should group based on unique sets of variants belonging to a certain cluster, egene, and credible set
e_nominal_pcsusie_subset['cluster_egene_credibleset'] = e_nominal_pcsusie_subset['phenotype_id'] + '_cs' + '_pc' + e_nominal_pcsusie_subset['cs_id'].str.split('_pc').str[-1]

# group by phenotype (egene and cluster)
pcphenotype_evariance = e_nominal_pcsusie_subset.groupby('cluster_egene_credibleset').agg({'variance_weighted':sum, 
                                                        'cluster_id':'first', 
                                                        'variant_id':list,
                                                        'egene_id':'first', 
                                                        'cs_id':'first', 
                                                        'lead_variant_id':'first',
                                                        'num_vars':'first',
                                                        'pc_num':'first'})

pcphenotype_evariance = pcphenotype_evariance.rename(columns={'variant_id':'variant_list'})

/local/scratch/klawren/slrmtmp.44431414/ipykernel_1284/2850190003.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_nominal_pcsusie_subset['var_cluster_egene'] = e_nominal_pcsusie_subset['variant_id'] + '_' + e_nominal_pcsusie_subset['phenotype_id']


In [117]:
e_nominal_pcsusie_subset.iloc[10]

phenotype_id                                                               NaN
variant_id                                                                 NaN
start_distance                                                             NaN
end_distance                                                               NaN
af                                                                         NaN
ma_samples                                                                 NaN
ma_count                                                                   NaN
pval_nominal                                                               NaN
slope                                                                      NaN
slope_se                                                                   NaN
cluster_id                                                                 NaN
var_cluster                  chr22_20084214_T_C_b38_ENSG00000099904.15_ENSG...
egene_id                                            

In [108]:
pcphenotype_evariance[pcphenotype_evariance['pc_num'] == 2]

,variance_weighted,cluster_id,variant_list,egene_id,cs_id,lead_variant_id,num_vars,pc_num
cluster_egene_credibleset,,,,,,,,


In [47]:
pcphenotype_evariance.head()

,variance_weighted,cluster_id,variant_list,egene_id,cs_id,lead_variant_id,num_vars
cluster_egene_credibleset,,,,,,,
ENSG00000025708.13_ENSG00000177989.13_e_ENSG00000025708.13_cs_pc1_1,0.057016,ENSG00000025708.13_ENSG00000177989.13,[chr22_50532837_T_C_b38],ENSG00000025708.13,ENSG00000025708.13_ENSG00000177989.13_pc1_1,chr22_50532837_T_C_b38,1
ENSG00000025708.13_ENSG00000177989.13_e_ENSG00000025708.13_cs_pc1_2,0.189943,ENSG00000025708.13_ENSG00000177989.13,"[chr22_50504763_C_T_b38, chr22_50505337_G_A_b3...",ENSG00000025708.13,ENSG00000025708.13_ENSG00000177989.13_pc1_2,chr22_50519627_C_T_b38,11
ENSG00000025708.13_ENSG00000177989.13_e_ENSG00000177989.13_cs_pc1_1,0.023457,ENSG00000025708.13_ENSG00000177989.13,[chr22_50532837_T_C_b38],ENSG00000177989.13,ENSG00000025708.13_ENSG00000177989.13_pc1_1,chr22_50532837_T_C_b38,1
ENSG00000025708.13_ENSG00000177989.13_e_ENSG00000177989.13_cs_pc1_2,0.269531,ENSG00000025708.13_ENSG00000177989.13,"[chr22_50504763_C_T_b38, chr22_50505337_G_A_b3...",ENSG00000177989.13,ENSG00000025708.13_ENSG00000177989.13_pc1_2,chr22_50519627_C_T_b38,11
ENSG00000093000.18_ENSG00000100364.18_e_ENSG00000093000.18_cs_pc1_1,0.350770,ENSG00000093000.18_ENSG00000100364.18,[chr22_45105543_G_A_b38],ENSG00000093000.18,ENSG00000093000.18_ENSG00000100364.18_pc1_1,chr22_45105543_G_A_b38,1


now I do something similar for the pc_nominals, basically just adding to the susie df the pc var explained

In [58]:
pc_nominal_pcsusie_subset

,phenotype_id,variant_id,start_distance,end_distance,af,ma_samples,ma_count,pval_nominal,slope,slope_se,...,var_cluster,var_pcgene_cluster,cs_id,pip,pc_num,lead_variant_id,num_vars,variance,variance_weighted,cluster_pc_credibleset
0,ENSG00000100033.16_ENSG00000183628.12_pc1,chr22_18933556_A_G_b38,27528,-2854,0.111801,104,108,1.005330e-13,0.040882,0.005308,...,chr22_18933556_A_G_b38_ENSG00000100033.16_ENSG...,chr22_18933556_A_G_b38_ENSG00000100033.16_ENSG...,ENSG00000100033.16_ENSG00000183628.12_pc1_1,0.939348,1,chr22_18933556_A_G_b38,2,0.167132,0.156995,ENSG00000100033.16_ENSG00000183628.12_pc1_cs_p...
1,ENSG00000100033.16_ENSG00000183628.12_pc2,chr22_18933556_A_G_b38,27528,-2854,0.111801,104,108,6.405509e-06,-0.025318,0.005538,...,chr22_18933556_A_G_b38_ENSG00000100033.16_ENSG...,chr22_18933556_A_G_b38_ENSG00000100033.16_ENSG...,ENSG00000100033.16_ENSG00000183628.12_pc1_1,0.939348,1,chr22_18933556_A_G_b38,2,0.064103,0.060215,ENSG00000100033.16_ENSG00000183628.12_pc2_cs_p...
2,ENSG00000100033.16_ENSG00000183628.12_pc1,chr22_18940813_T_C_b38,34785,4403,0.110766,104,107,9.494485e-13,0.039920,0.005418,...,chr22_18940813_T_C_b38_ENSG00000100033.16_ENSG...,chr22_18940813_T_C_b38_ENSG00000100033.16_ENSG...,ENSG00000100033.16_ENSG00000183628.12_pc1_1,0.060652,1,chr22_18933556_A_G_b38,2,0.159358,0.009665,ENSG00000100033.16_ENSG00000183628.12_pc1_cs_p...
3,ENSG00000100033.16_ENSG00000183628.12_pc2,chr22_18940813_T_C_b38,34785,4403,0.110766,104,107,1.865919e-06,-0.027117,0.005606,...,chr22_18940813_T_C_b38_ENSG00000100033.16_ENSG...,chr22_18940813_T_C_b38_ENSG00000100033.16_ENSG...,ENSG00000100033.16_ENSG00000183628.12_pc1_1,0.060652,1,chr22_18933556_A_G_b38,2,0.073531,0.004460,ENSG00000100033.16_ENSG00000183628.12_pc2_cs_p...
4,ENSG00000099904.15_ENSG00000234409.6_pc1,chr22_20153539_C_A_b38,24083,2474,0.319876,261,309,1.132345e-48,0.048625,0.002893,...,chr22_20153539_C_A_b38_ENSG00000099904.15_ENSG...,chr22_20153539_C_A_b38_ENSG00000099904.15_ENSG...,ENSG00000099904.15_ENSG00000234409.6_pc1_1,0.142057,1,chr22_20154469_C_G_b38,3,0.236434,0.033587,ENSG00000099904.15_ENSG00000234409.6_pc1_cs_pc1_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,ENSG00000025708.13_ENSG00000177989.13_pc2,chr22_50535371_C_G_b38,5315,2791,0.021739,20,21,5.438037e-01,0.007987,0.013145,...,chr22_50535371_C_G_b38_ENSG00000025708.13_ENSG...,chr22_50535371_C_G_b38_ENSG00000025708.13_ENSG...,ENSG00000025708.13_ENSG00000177989.13_pc1_2,0.101850,1,chr22_50519627_C_T_b38,11,0.006379,0.000650,ENSG00000025708.13_ENSG00000177989.13_pc2_cs_p...
200,ENSG00000025708.13_ENSG00000177989.13_pc1,chr22_50540263_G_A_b38,10207,7683,0.022774,20,22,5.443152e-06,-0.055229,0.011987,...,chr22_50540263_G_A_b38_ENSG00000025708.13_ENSG...,chr22_50540263_G_A_b38_ENSG00000025708.13_ENSG...,ENSG00000025708.13_ENSG00000177989.13_pc1_2,0.088870,1,chr22_50519627_C_T_b38,11,0.305021,0.027107,ENSG00000025708.13_ENSG00000177989.13_pc1_cs_p...
201,ENSG00000025708.13_ENSG00000177989.13_pc2,chr22_50540263_G_A_b38,10207,7683,0.022774,20,22,3.925436e-01,0.010511,0.012281,...,chr22_50540263_G_A_b38_ENSG00000025708.13_ENSG...,chr22_50540263_G_A_b38_ENSG00000025708.13_ENSG...,ENSG00000025708.13_ENSG00000177989.13_pc1_2,0.088870,1,chr22_50519627_C_T_b38,11,0.011048,0.000982,ENSG00000025708.13_ENSG00000177989.13_pc2_cs_p...
202,ENSG00000025708.13_ENSG00000177989.13_pc1,chr22_50541096_C_A_b38,11040,8516,0.021739,20,21,6.125603e-06,-0.058775,0.012829,...,chr22_50541096_C_A_b38_ENSG00000025708.13_ENSG...,chr22_50541096_C_A_b38_ENSG00000025708.13_ENSG...,ENSG00000025708.13_ENSG00000177989.13_pc1_2,0.101850,1,chr22_50519627_C_T_b38,11,0.345452,0.035184,ENSG00000025708.13_ENSG00000177989.13_pc1_cs_p...


In [65]:
# get a subset of variants that ended up in credible sets
pc_nominal_pcsusie_subset = pc_nominal[pc_nominal['var_cluster'].isin(pc_susie['var_cluster'])]

# I want to get a pip weighted variance for each egene
pc_nominal_pcsusie_subset['var_pcgene_cluster'] = pc_nominal_pcsusie_subset['variant_id'] + '_' + pc_nominal_pcsusie_subset['phenotype_id']

# merge in the data from the susie finemapping
pc_nominal_pcsusie_subset = pd.merge(pc_nominal_pcsusie_subset, pc_susie[['cs_id', 'var_cluster', 'pip', 'pc_num', 'lead_variant_id', 'num_vars']], on='var_cluster')

# pip weighted variance
pc_nominal_pcsusie_subset['variance'] = pc_nominal_pcsusie_subset['slope'].apply(np.square) * 100
pc_nominal_pcsusie_subset['variance_weighted'] = pc_nominal_pcsusie_subset['variance'] * e_nominal_pcsusie_subset['pip'] 

# we should group based on unique sets of variants belonging to a certain cluster, and credible set
# because it only makes sese to look at vars for the pc that the credible set is for, we drop the irrelevant pc-pc pairings
pc_nominal_pcsusie_subset['cluster_pc_credibleset'] = pc_nominal_pcsusie_subset['phenotype_id'] + '_cs' + '_pc' + pc_nominal_pcsusie_subset['cs_id'].str.split('_pc').str[-1]

# group by phenotype (egene and cluster)
pcphenotype_pcvariance = pc_nominal_pcsusie_subset.groupby('cluster_pc_credibleset').agg({'variance_weighted':sum, 
                                                        'cluster_id':'first', 
                                                        'variant_id':list,
                                                        'cs_id':'first', 
                                                        'lead_variant_id':'first',
                                                        'num_vars':'first',
                                                        'pc_num':list})

pcphenotype_pcvariance = pcphenotype_pcvariance.rename(columns={'variant_id':'variant_list'})


/local/scratch/klawren/slrmtmp.44431414/ipykernel_1284/787166930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pc_nominal_pcsusie_subset['var_pcgene_cluster'] = pc_nominal_pcsusie_subset['variant_id'] + '_' + pc_nominal_pcsusie_subset['phenotype_id']


In [69]:
pc_nominal_pcsusie_subset['pc_num_variant'] = pc_nominal_pcsusie_subset['pc_num']
pc_nominal_pcsusie_subset['pc_num_cs'] = pc_nominal_pcsusie_subset['pc_num']

,phenotype_id,variant_id,start_distance,end_distance,af,ma_samples,ma_count,pval_nominal,slope,slope_se,...,var_cluster,var_pcgene_cluster,cs_id,pip,pc_num,lead_variant_id,num_vars,variance,variance_weighted,cluster_pc_credibleset
0,ENSG00000100033.16_ENSG00000183628.12_pc1,chr22_18933556_A_G_b38,27528,-2854,0.111801,104,108,1.005330e-13,0.040882,0.005308,...,chr22_18933556_A_G_b38_ENSG00000100033.16_ENSG...,chr22_18933556_A_G_b38_ENSG00000100033.16_ENSG...,ENSG00000100033.16_ENSG00000183628.12_pc1_1,0.939348,1,chr22_18933556_A_G_b38,2,0.167132,0.156995,ENSG00000100033.16_ENSG00000183628.12_pc1_cs_p...
1,ENSG00000100033.16_ENSG00000183628.12_pc2,chr22_18933556_A_G_b38,27528,-2854,0.111801,104,108,6.405509e-06,-0.025318,0.005538,...,chr22_18933556_A_G_b38_ENSG00000100033.16_ENSG...,chr22_18933556_A_G_b38_ENSG00000100033.16_ENSG...,ENSG00000100033.16_ENSG00000183628.12_pc1_1,0.939348,1,chr22_18933556_A_G_b38,2,0.064103,0.060215,ENSG00000100033.16_ENSG00000183628.12_pc2_cs_p...
2,ENSG00000100033.16_ENSG00000183628.12_pc1,chr22_18940813_T_C_b38,34785,4403,0.110766,104,107,9.494485e-13,0.039920,0.005418,...,chr22_18940813_T_C_b38_ENSG00000100033.16_ENSG...,chr22_18940813_T_C_b38_ENSG00000100033.16_ENSG...,ENSG00000100033.16_ENSG00000183628.12_pc1_1,0.060652,1,chr22_18933556_A_G_b38,2,0.159358,0.009665,ENSG00000100033.16_ENSG00000183628.12_pc1_cs_p...
3,ENSG00000100033.16_ENSG00000183628.12_pc2,chr22_18940813_T_C_b38,34785,4403,0.110766,104,107,1.865919e-06,-0.027117,0.005606,...,chr22_18940813_T_C_b38_ENSG00000100033.16_ENSG...,chr22_18940813_T_C_b38_ENSG00000100033.16_ENSG...,ENSG00000100033.16_ENSG00000183628.12_pc1_1,0.060652,1,chr22_18933556_A_G_b38,2,0.073531,0.004460,ENSG00000100033.16_ENSG00000183628.12_pc2_cs_p...
4,ENSG00000099904.15_ENSG00000234409.6_pc1,chr22_20153539_C_A_b38,24083,2474,0.319876,261,309,1.132345e-48,0.048625,0.002893,...,chr22_20153539_C_A_b38_ENSG00000099904.15_ENSG...,chr22_20153539_C_A_b38_ENSG00000099904.15_ENSG...,ENSG00000099904.15_ENSG00000234409.6_pc1_1,0.142057,1,chr22_20154469_C_G_b38,3,0.236434,0.033587,ENSG00000099904.15_ENSG00000234409.6_pc1_cs_pc1_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,ENSG00000025708.13_ENSG00000177989.13_pc2,chr22_50535371_C_G_b38,5315,2791,0.021739,20,21,5.438037e-01,0.007987,0.013145,...,chr22_50535371_C_G_b38_ENSG00000025708.13_ENSG...,chr22_50535371_C_G_b38_ENSG00000025708.13_ENSG...,ENSG00000025708.13_ENSG00000177989.13_pc1_2,0.101850,1,chr22_50519627_C_T_b38,11,0.006379,0.000650,ENSG00000025708.13_ENSG00000177989.13_pc2_cs_p...
200,ENSG00000025708.13_ENSG00000177989.13_pc1,chr22_50540263_G_A_b38,10207,7683,0.022774,20,22,5.443152e-06,-0.055229,0.011987,...,chr22_50540263_G_A_b38_ENSG00000025708.13_ENSG...,chr22_50540263_G_A_b38_ENSG00000025708.13_ENSG...,ENSG00000025708.13_ENSG00000177989.13_pc1_2,0.088870,1,chr22_50519627_C_T_b38,11,0.305021,0.027107,ENSG00000025708.13_ENSG00000177989.13_pc1_cs_p...
201,ENSG00000025708.13_ENSG00000177989.13_pc2,chr22_50540263_G_A_b38,10207,7683,0.022774,20,22,3.925436e-01,0.010511,0.012281,...,chr22_50540263_G_A_b38_ENSG00000025708.13_ENSG...,chr22_50540263_G_A_b38_ENSG00000025708.13_ENSG...,ENSG00000025708.13_ENSG00000177989.13_pc1_2,0.088870,1,chr22_50519627_C_T_b38,11,0.011048,0.000982,ENSG00000025708.13_ENSG00000177989.13_pc2_cs_p...
202,ENSG00000025708.13_ENSG00000177989.13_pc1,chr22_50541096_C_A_b38,11040,8516,0.021739,20,21,6.125603e-06,-0.058775,0.012829,...,chr22_50541096_C_A_b38_ENSG00000025708.13_ENSG...,chr22_50541096_C_A_b38_ENSG00000025708.13_ENSG...,ENSG00000025708.13_ENSG00000177989.13_pc1_2,0.101850,1,chr22_50519627_C_T_b38,11,0.345452,0.035184,ENSG00000025708.13_ENSG00000177989.13_pc1_cs_p...


In [68]:
pcphenotype_pcvariance['pc_num']

cluster_pc_credibleset
ENSG00000025708.13_ENSG00000177989.13_pc1_cs_pc1_1                                                                                      [1]
ENSG00000025708.13_ENSG00000177989.13_pc1_cs_pc1_2                                                        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
ENSG00000025708.13_ENSG00000177989.13_pc2_cs_pc1_1                                                                                      [1]
ENSG00000025708.13_ENSG00000177989.13_pc2_cs_pc1_2                                                        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
ENSG00000093000.18_ENSG00000100364.18_pc1_cs_pc1_1                                                                                      [1]
ENSG00000093000.18_ENSG00000100364.18_pc2_cs_pc1_1                                                                                      [1]
ENSG00000099904.15_ENSG00000234409.6_pc1_cs_pc1_1                                                                                 [1, 1, 

In [62]:
pcphenotype_pcvariance['cluster_pc_credibleset'].str.split['_pc'].str[0]

KeyError: 'cluster_pc_credibleset'

In [20]:
pd.merge(pcphenotype_pcvariance, pcphenotype_evariance, on = 'cs_id')

,variance_weighted_x,cluster_id_x,variant_id_x,cs_id,variance_weighted_y,cluster_id_y,variant_id_y,egene_id
0,0.381875,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000025708.13_ENSG00000177989.13_2,0.246960,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000025708.13
1,0.381875,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000025708.13_ENSG00000177989.13_2,0.292988,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000177989.13
2,0.011984,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000025708.13_ENSG00000177989.13_2,0.246960,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000025708.13
3,0.011984,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000025708.13_ENSG00000177989.13_2,0.292988,ENSG00000025708.13_ENSG00000177989.13,chr22_50504763_C_T_b38,ENSG00000177989.13
4,0.294142,ENSG00000093000.18_ENSG00000100364.18,chr22_45105543_G_A_b38,ENSG00000093000.18_ENSG00000100364.18_1,0.350770,ENSG00000093000.18_ENSG00000100364.18,chr22_45105543_G_A_b38,ENSG00000093000.18
...,...,...,...,...,...,...,...,...
63,0.019338,ENSG00000183066.14_ENSG00000198951.11,chr22_41992822_A_G_b38,ENSG00000183066.14_ENSG00000198951.11_1,0.252443,ENSG00000183066.14_ENSG00000198951.11,chr22_41992822_A_G_b38,ENSG00000198951.11
64,0.268754,ENSG00000243811.8_ENSG00000244509.3,chr22_39023260_C_T_b38,ENSG00000243811.8_ENSG00000244509.3_1,0.230780,ENSG00000243811.8_ENSG00000244509.3,chr22_39023260_C_T_b38,ENSG00000243811.8
65,0.268754,ENSG00000243811.8_ENSG00000244509.3,chr22_39023260_C_T_b38,ENSG00000243811.8_ENSG00000244509.3_1,0.124338,ENSG00000243811.8_ENSG00000244509.3,chr22_39023260_C_T_b38,ENSG00000244509.3
66,0.012858,ENSG00000243811.8_ENSG00000244509.3,chr22_39023260_C_T_b38,ENSG00000243811.8_ENSG00000244509.3_1,0.230780,ENSG00000243811.8_ENSG00000244509.3,chr22_39023260_C_T_b38,ENSG00000243811.8


I now have a table with a row for the % evar explained for each egene and one with the % pcvar for the pc-overall. I want to further annotate this with relevant info, like the pc-qtl lead variant to egene tss distance

In [11]:
# load in the gene information (start and strand are what I need)
full_gencode=pd.read_csv('/home/klawren/oak/pcqtls/data/references/processed_gencode.v26.GRCh38.genes.gtf', sep='\t', skiprows=range(6), 
            header=None, names=['chr', 'dataset', 'type', 'start','end', '.', 'strand', 'na', 'info'])

full_gencode = full_gencode[full_gencode['type']=='transcript']
full_gencode['transcript_id'] = full_gencode['info'].str.split(';').str[1].str.split('\"').str[-2]

# add in the start and end info
full_gencode['tss_start'] = np.where(full_gencode['strand'] == '+', full_gencode['start'], full_gencode['end'])
full_gencode['gene_end'] = np.where(full_gencode['strand'] == '-', full_gencode['start'], full_gencode['end'])

# filter to just the transcripts that are in the clusters
gene_ids = np.concatenate(overlap_df['cluster_id'].str.split('_'))
gid_gencode = full_gencode.set_index('transcript_id').loc[gene_ids]
gid_gencode = gid_gencode.drop_duplicates()

In [15]:
row = sum_variance_df.iloc[0]

In [19]:
egene_tss = gid_gencode.loc[row['egene_id']]['tss_start']
lead

KeyError: 'egene_id'